# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
G1 = nx.read_gpickle("assets/A4_P1_G1")
G2 = nx.read_gpickle("assets/A4_P1_G2")
G3 = nx.read_gpickle("assets/A4_P1_G3")
G4 = nx.read_gpickle("assets/A4_P1_G4")
G5 = nx.read_gpickle("assets/A4_P1_G5")
P1_Graphs = [G1, G2, G3, G4, G5]

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs using any methodology and determine which of the three algorithms generated each graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [164]:
def graph_identification():
    # YOUR CODE HERE
    def degree_dist(graph):
        degrees = graph.degree()
        degree_val = sorted(set(val for (node, val) in degrees))
        histogram = [list(val for (node, val) in degrees).count(i)/float(nx.number_of_nodes(graph)) for i in degree_val]
        return histogram

    algorithm = []

    for graph in P1_Graphs:
        clustering = nx.average_clustering(graph)
        shortest_path = nx.average_shortest_path_length(graph)
        # degree_hist = degree_dist(graph)
        degree_hist = nx.degree_histogram(graph)
        
        if len(degree_hist) > 10:
            algorithm.append('PA')
        elif clustering < .4:
            algorithm.append('SW_H')
        else:
            algorithm.append('SW_L')
    return algorithm
print(graph_identification())


['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']


In [5]:
ans_one = graph_identification()
assert type(ans_one) == list, "You must return a list"


---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [6]:
G = pickle.load(open('assets/email_prediction_NEW.txt', 'rb'))

print(f"Graph with {len(nx.nodes(G))} nodes and {len(nx.edges(G))} edges")

Graph with 1005 nodes and 16706 edges


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features of your choice using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a Pandas series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [7]:
list(G.nodes(data=True))[:5] # print the first 5 nodes

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (581, {'Department': 3, 'ManagementSalary': 0.0}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (65, {'Department': 4, 'ManagementSalary': nan})]

In [151]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
def salary_predictions():
    # Loading node data into a dataframe.
    node_data = np.array(G.nodes(data=True))

    df = pd.DataFrame({'ManagementSalary': [d['ManagementSalary'] for d in node_data[:, 1]],
                   'department': [d['Department'] for d in node_data[:, 1]]},
                  index=node_data[:, 0])
                  
# Getting metrics to use in classifier.
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Degree'] = pd.Series(val for (node, val) in G.degree())
    df['Degree Centrality'] = pd.Series(nx.degree_centrality(G))
    df['Closeness'] = pd.Series(nx.closeness_centrality(G))
    df['Betweenness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['Page Rank'] = pd.Series(nx.pagerank(G))

# Setting up train and test sets.
    train =  df[~df['ManagementSalary'].isnull()]
    test = df[df['ManagementSalary'].isnull()]

    X_train = train.iloc[:,1:]
    X_test = test.iloc[:,1:]

    y_train = train['ManagementSalary']
# Fitting the GBC
    from sklearn.ensemble import GradientBoostingClassifier

    clf = GradientBoostingClassifier(random_state=0)
    clf.fit(X_train, y_train)
# Getting probabilities
    test_prob = pd.Series(clf.predict_proba(X_test)[:, 1], index= X_test.index)

    return test_prob
salary_predictions()

1      0.005663
65     0.984548
18     0.023670
215    0.972557
283    0.986396
         ...   
691    0.001416
788    0.000939
944    0.001513
798    0.001189
808    0.001224
Length: 252, dtype: float64

In [99]:
ans_salary_preds = salary_predictions()
assert type(ans_salary_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_salary_preds) == 252, "The series must be of length 252"


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [13]:
future_connections = pd.read_csv('assets/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to:      
1. Create a matrix of features of your choice for the edges found in `future_connections` using Networkx     
2. Train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data     
3. Predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [14]:
def new_connections_predictions():
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier

    future_connections['preferential_attachment'] = [list(nx.preferential_attachment(G, [node_pair]))[0][2]
                                             for node_pair in future_connections.index]
    future_connections['common_neighbors'] = [len(list(nx.common_neighbors(G, node_pair[0], node_pair[1]))) 
                                            for node_pair in future_connections.index]
    train_data = future_connections[~future_connections['Future Connection'].isnull()]
    test_data = future_connections[future_connections['Future Connection'].isnull()]
    
    rfc = RandomForestClassifier(random_state=24, n_estimators=100, max_leaf_nodes=20, min_samples_split=15, criterion = 'entropy')
    rfc.fit(train_data[['preferential_attachment','common_neighbors']].values, train_data['Future Connection'].values)
    preds = rfc.predict_proba(test_data[['preferential_attachment','common_neighbors']].values)[:,1]
    return pd.Series(preds, index=test_data.index)
new_connections_predictions()

(107, 348)    0.000000
(542, 751)    0.013506
(20, 426)     0.000000
(50, 989)     0.013770
(942, 986)    0.011023
                ...   
(165, 923)    0.012389
(673, 755)    0.010060
(939, 940)    0.011023
(555, 905)    0.020177
(75, 101)     0.000000
Length: 122112, dtype: float64

In [ ]:
ans_prob_preds = new_connections_predictions()
assert type(ans_prob_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_prob_preds) == 122112, "The series must be of length 122112"
